In [22]:
import ee
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
%matplotlib inline

from IPython.display import Image

In [6]:
# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/1AfgeXvtE7ilw6oCTG17nFMacUjeKUM8HtzMxAJZ96JmMvK9yiGQcemOOjIo

Successfully saved authorization token.


In [7]:
# Initialize the library.
ee.Initialize()

In [29]:
# Import the LANDSAT surface reflectance colelction.
ls_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [42]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

In [43]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

ls_sr = ls_sr.select('SR_B4', 'SR_B3', 'SR_B2').filterBounds(u_poi).filterDate(i_date, f_date)

In [44]:
first_image = ls_sr.first()
first_image.bandNames().getInfo()

['SR_B4', 'SR_B3', 'SR_B2']

In [48]:
first_image = first_image.toFloat()

In [70]:
roi = u_poi.buffer(distance=9.2e5)
dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
dataset = dataset.select('SR_B4', 'SR_B3', 'SR_B2')

date_filter = ee.Filter.date('2021-05-01', '2021-06-01')
dataset = dataset.filter(date_filter)

# def applyScaleFactors(image):
#     opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
#     thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
#     return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)


# dataset = dataset.map(applyScaleFactors);

image = dataset.mean()

visualisation_params = { 
  'min': 8,
  'max': 40,
  'dimensions': 620,
  'region': roi,
}
url = image.getThumbUrl(visualisation_params)

Image(url=url)

In [ ]:
ee.Initialize()

poi = ee.Geometry.Point([122.1247, 13.2735]) # [long, lat]
roi = poi.buffer(distance=9.2e5)

dataset = ee.ImageCollection("MODIS/006/MOD11A1")
dataset = dataset.select('LST_Day_1km')

date_filter = ee.Filter.date('2021-01-01', '2022-01-01')
dataset = dataset.filter(date_filter)

image = dataset.mean()
image = image.multiply(0.02).add(-273.15)
visualisation_params = { 
  'min': 8,
  'max': 40,
  'dimensions': 620,
  'region': roi,
  'palette': ['blue', 'yellow', 'red']
}
url = image.getThumbUrl(visualisation_params)

Image(url=url)